In [1]:
from keras.applications import vgg19
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
from skimage import io,transform
import os
import sys
model_name = 'VGGNet_Incident'

Using TensorFlow backend.


In [ ]:
# counter = 0
# folder = '../Images/Incident/'
# data = pd.read_csv(os.path.join(folder, 'labels.csv'))
# imageFiles = os.listdir(folder)
# total = len(imageFiles)
# images = []
# labels = []
# size = 128
# for image_file in imageFiles:
#     if not image_file.endswith('.jpg'):
#         continue
#     img = io.imread(os.path.join(folder,image_file))
#     img = transform.resize(img, (size,size,3))
#     image_file = image_file.split('.')[0]
#     images.append(img)
#     labels.append(data[data.tid == float(image_file)].values[0][1:])
#     counter += 1
#     sys.stdout.write("\rProcessed: {0}/{1}".format(counter, total))
#     sys.stdout.flush()

# x = np.array(images)
# y = np.array(labels)


# np.save('../Images/Incident/incident_x%s.npy' % size, x)
# np.save('../Images/Incident/incident_y%s.npy' % size, y)

In [2]:
size = 128
x = np.load('../Images/Incident/incident_x128.npy')
y = np.load('../Images/Incident/incident_y128.npy')

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

In [4]:
import keras.backend as K
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [5]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [8]:
from keras.layers import GlobalAveragePooling2D, Dropout, Dense
from keras.models import Model
base = vgg19.VGG19(include_top=False, weights=None, classes=3, input_shape=(size,size,3), pooling='avg')
temp = base.output
temp = Dropout(0.5)(temp)
predictions = Dense(3, activation= 'sigmoid')(temp)
model = Model(inputs = base.input, outputs = predictions)

In [9]:
from keras.callbacks import ModelCheckpoint
# rmsp = rmsprop(lr=0.01, decay=0.9, epsilon=1, clipvalue=2)
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['acc'])

filepath="%s.best.hdf5" % model_name
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

hist = model.fit([x_train], y_train, batch_size=32, epochs=100, validation_data=([x_test], y_test), verbose=1, callbacks=callbacks_list)

Train on 630 samples, validate on 158 samples
Epoch 1/100
630/630 [==============================] - 3s 5ms/step - loss: 0.6885 - acc: 0.7201 - val_loss: 0.6830 - val_acc: 0.7468

Epoch 00001: val_loss improved from inf to 0.68300, saving model to VGGNet_Incident.best.hdf5
Epoch 2/100
630/630 [==============================] - 3s 4ms/step - loss: 0.6786 - acc: 0.7423 - val_loss: 0.6730 - val_acc: 0.7468

Epoch 00002: val_loss improved from 0.68300 to 0.67301, saving model to VGGNet_Incident.best.hdf5
Epoch 3/100
630/630 [==============================] - 3s 4ms/step - loss: 0.6691 - acc: 0.7423 - val_loss: 0.6634 - val_acc: 0.7468

Epoch 00003: val_loss improved from 0.67301 to 0.66336, saving model to VGGNet_Incident.best.hdf5
Epoch 4/100
630/630 [==============================] - 3s 4ms/step - loss: 0.6600 - acc: 0.7423 - val_loss: 0.6541 - val_acc: 0.7468

Epoch 00004: val_loss improved from 0.66336 to 0.65413, saving model to VGGNet_Incident.best.hdf5
Epoch 5/100
630/630 [=========

Epoch 37/100
630/630 [==============================] - 3s 4ms/step - loss: 0.5551 - acc: 0.7423 - val_loss: 0.5468 - val_acc: 0.7468

Epoch 00037: val_loss did not improve from 0.54675
Epoch 38/100
630/630 [==============================] - 3s 4ms/step - loss: 0.5561 - acc: 0.7423 - val_loss: 0.5469 - val_acc: 0.7468

Epoch 00038: val_loss did not improve from 0.54675
Epoch 39/100
630/630 [==============================] - 3s 4ms/step - loss: 0.5562 - acc: 0.7423 - val_loss: 0.5468 - val_acc: 0.7468

Epoch 00039: val_loss did not improve from 0.54675
Epoch 40/100
630/630 [==============================] - 3s 4ms/step - loss: 0.5554 - acc: 0.7423 - val_loss: 0.5467 - val_acc: 0.7468

Epoch 00040: val_loss improved from 0.54675 to 0.54668, saving model to VGGNet_Incident.best.hdf5
Epoch 41/100
630/630 [==============================] - 3s 4ms/step - loss: 0.5567 - acc: 0.7423 - val_loss: 0.5466 - val_acc: 0.7468

Epoch 00041: val_loss improved from 0.54668 to 0.54665, saving model to VG

630/630 [==============================] - 3s 4ms/step - loss: 0.5442 - acc: 0.7423 - val_loss: 0.5344 - val_acc: 0.7468

Epoch 00073: val_loss improved from 0.53610 to 0.53435, saving model to VGGNet_Incident.best.hdf5
Epoch 74/100
630/630 [==============================] - 3s 4ms/step - loss: 0.5416 - acc: 0.7423 - val_loss: 0.5334 - val_acc: 0.7468

Epoch 00074: val_loss improved from 0.53435 to 0.53339, saving model to VGGNet_Incident.best.hdf5
Epoch 75/100
630/630 [==============================] - 3s 4ms/step - loss: 0.5390 - acc: 0.7423 - val_loss: 0.5300 - val_acc: 0.7468

Epoch 00075: val_loss improved from 0.53339 to 0.52999, saving model to VGGNet_Incident.best.hdf5
Epoch 76/100
630/630 [==============================] - 3s 4ms/step - loss: 0.5377 - acc: 0.7423 - val_loss: 0.5312 - val_acc: 0.7468

Epoch 00076: val_loss did not improve from 0.52999
Epoch 77/100
630/630 [==============================] - 3s 4ms/step - loss: 0.5399 - acc: 0.7423 - val_loss: 0.5377 - val_acc: 0

In [10]:
# serialize model to JSON
model_json = model.to_json()
with open("%s.json" % model_name, "w") as json_file:
    json_file.write(model_json)
    
# # serialize weights to HDF5
model.save_weights("%s.h5" % model_name)
print("Saved model to disk")

Saved model to disk


In [ ]:
from keras.models import model_from_json
# load json and create model
json_file = open('%s.json' % model_name, 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("%s.h5" % model_name)
print("Loaded model from disk")

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['acc'])

In [11]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, roc_auc_score
predicted = model.predict([x_test])
print(classification_report(y_test, predicted > 0.5))
print(precision_score(y_test, predicted > 0.5, average='micro'))
print(recall_score(y_test, predicted > 0.5, average='micro'))
print(roc_auc_score(y_test, predicted > 0.5, average='micro'))
print(f1_score(y_test, predicted > 0.5, average='micro'))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        22
          1       0.00      0.00      0.00        51
          2       0.70      1.00      0.83       111

avg / total       0.42      0.60      0.50       184

0.7025316455696202
0.6032608695652174
0.720595952023988
0.6491228070175438


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
from keras.models import model_from_json
# load json and create model
json_file = open('%s.json' % model_name, 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("%s.best.hdf5" % model_name)
print("Loaded model from disk")

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['acc'])

Loaded model from disk


In [13]:
predicted = model.predict([x_test])
print(classification_report(y_test, predicted > 0.5))
print(precision_score(y_test, predicted > 0.5, average='micro'))
print(recall_score(y_test, predicted > 0.5, average='micro'))
print(roc_auc_score(y_test, predicted > 0.5, average='micro'))
print(f1_score(y_test, predicted > 0.5, average='micro'))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        22
          1       0.00      0.00      0.00        51
          2       0.70      1.00      0.83       111

avg / total       0.42      0.60      0.50       184

0.7025316455696202
0.6032608695652174
0.720595952023988
0.6491228070175438


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
